In [1]:
from jeremy import spGrids
import numpy as np

In [2]:
A = np.array([[-1,1,1],[1,-1,1],[1,1,-1]])
H = np.array([[2,0,0],[1,2,0],[1,1,2]])
B = np.dot(A,H)
G = np.transpose(np.linalg.inv(B))

In [3]:
def make_cell_points(HNF_diags,lat_vecs,grid_vecs):
    """Makes the points of grid inside the volume of the lattice vecs.
    
    Args:
        HNF_diags (list of int): The diagonals of the HNF.
        grid_vecs (numpy ndarray): A matrix whose columns hold the basis vectors 
            of the lattice, in cartesian coordinates.
        grid_vecs (numpy ndarray): A matrix whose columns hold the basis vectors 
            of the grid, in cartesian coordinates.
            
    Returns:
        points (numpy ndarray): The points of the grid inside the lattice expressed in
            lattice coordinates.
    """
    
    v1 = grid_vecs[:,0]
    v2 = grid_vecs[:,1]
    v3 = grid_vecs[:,2]
    
    points = []
    for i in range(HNF_diags[0]):
        for j in range(HNF_diags[1]):
            for k in range(HNF_diags[2]):
                points.append(np.dot(np.linalg.inv(lat_vecs),v1*i+v2*j+v3*k))

    points = np.array(points)%1
    return np.array([np.dot(lat_vecs,i) for i in points])

In [4]:
def get_ijk(G,point):
    """Finds the integer multiple of the grid vectors to reach the desired point.
    
    Args:
        G (numpy.ndarray): The grid vectors as columns of a matrix.
        point (numpy.ndarray): The point to in question.
        
    Returns:
        ijk (list of int): The integers needed to multiply the grid vectors by to make 
            the point.
    """
    
    ijk = np.dot(np.linalg.inv(G),point)

    a1 = G[:,0]
    a2 = G[:,1]
    a3 = G[:,2]
    
    return tuple(ijk)

In [34]:
def find_irreducible_points(grid_points,parent,grid_vecs,offset=None):
    """Finds the irreducible grid points from the original grid.
    
    Args:
        grid_points (numpy.ndarray): The array of points in cartesian coordinates.
        parent (numpy.ndarray): The parent lattice vectors as columns of a matrix.
        grid_vecs (numpy.ndarray): The generating vectors of the grid as columns of a matrix.
        offset (optional, float): The grid offset as fractions of the parent lattice.
        
    Returns:
        irreducible_points (numpy.ndarray): The irreducibel points in the array and their weights.
    """
    
    from phenum.symmetry import _get_lattice_pointGroup
    
    if offset is None:
        offset = np.array([0,0,0])
    else:
        offset = np.dot(parent,offset)
    
    pgOps = _get_lattice_pointGroup(np.transpose(parent))
    
    hash_list = {}
    grid_points = np.array([point+offset for point in grid_points])
    grid_points = np.array([np.dot(np.linalg.inv(parent),point) for point in grid_points])%1
    grid_points = np.array([np.dot(parent,point) for point in grid_points])
    for p in grid_points:
        hash_list[hash(get_ijk(grid_vecs,p))] = 0
    
    irreducibel_points = []
    
    for p in grid_points:
        weight = 1
        if hash_list[hash(get_ijk(grid_vecs,p))] == 0:
            for Op in pgOps:
                new_p = np.dot(Op,p)
                new_p = np.dot(np.linalg.inv(parent),new_p)%1
                new_p = np.dot(parent,new_p)
                new_hash = hash(get_ijk(grid_vecs,new_p))
                if new_hash in hash_list and new_hash != hash(get_ijk(grid_vecs,p)):
                    if hash_list[new_hash] != 0:
                        break
                    else:
                        hash_list[new_hash] = 1
                        weight += 1
            irreducibel_points.append([p,weight])
        
    return irreducibel_points                

In [110]:
G = spGrids(A,216)
G = G[0]
G = G["grid_vecs"]
pnts = make_cell_points([6,6,6],np.transpose(np.linalg.inv(A)),G)
irpnts=find_irreducible_points(pnts,np.transpose(np.linalg.inv(A)),G)#,offset=np.array([0.5,0.5,0.5]))

('name', 'fcc')
('HNF', [[6, 0, 0], [0, 6, 0], [0, 0, 6]])


In [111]:
len(pnts)/float(len(irpnts))

1.6875

In [112]:
len(irpnts)

128

In [113]:
irpnts

[[array([ 0.,  0.,  0.]), 1],
 [array([ 0.        ,  0.08333333,  0.08333333]), 2],
 [array([ 0.        ,  0.16666667,  0.16666667]), 2],
 [array([ 0.  ,  0.25,  0.25]), 5],
 [array([ 0.5       ,  0.08333333,  0.41666667]), 4],
 [array([ 0.5       ,  0.16666667,  0.5       ]), 2],
 [array([ 0.5       ,  0.25      ,  0.58333333]), 2],
 [array([ 0.5       ,  0.33333333,  0.66666667]), 3],
 [array([ 0.5       ,  0.41666667,  0.75      ]), 3],
 [array([ 0.5       ,  0.5       ,  0.83333333]), 2],
 [array([ 0.5       ,  0.16666667,  0.33333333]), 3],
 [array([ 0.5       ,  0.25      ,  0.41666667]), 3],
 [array([ 0.5       ,  0.33333333,  0.5       ]), 2],
 [array([ 0.5       ,  0.41666667,  0.58333333]), 2],
 [array([ 0.5       ,  0.5       ,  0.66666667]), 1],
 [array([ 0.5 ,  0.25,  0.25]), 1],
 [array([ 0.5       ,  0.33333333,  0.33333333]), 1],
 [array([ 0.5       ,  0.41666667,  0.41666667]), 4],
 [array([ 0.5,  0.5,  0.5]), 1],
 [array([ 0.5       ,  0.58333333,  0.58333333]), 2],
 

In [114]:
len(pnts)

216